## Data Info
Getting the data info such as data size, number of feature, number of records, mean value, max value, etc....

In [14]:
import pandas as pd

In [15]:
crime_data = pd.read_csv('Crimes_2001_to_Present.csv')

In [16]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734075 entries, 0 to 734074
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            734075 non-null  int64  
 1   ID                    734075 non-null  int64  
 2   New_Date              734075 non-null  object 
 3   Primary Type          734075 non-null  object 
 4   Location Description  730589 non-null  object 
 5   Arrest                734075 non-null  int64  
 6   Community Area        734074 non-null  float64
 7   Location              723460 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 44.8+ MB


## Converting the Arrest Feature data type
bool -> int

In [ ]:
crime_data['Arrest'] = crime_data['Arrest'].replace({
    True:1,
    False:0
})

crime_data['Arrest'].unique()

## Converting the Date


In [ ]:

crime_data['New_Date'] = pd.to_datetime(crime_data['Date'])
# crime_data['New_Date'] = crime_data['New_Date'].dt.strftime('%m-%d-%Y %H:%M:%S')

In [ ]:
# comparing the old date value format to the datetime value format
crime_data[['Date', 'New_Date']]

In [ ]:
print(crime_data[crime_data['New_Date'].dt.year >= 2021].count()) # How many records we have from 2021 to 2024
print("Max new_date value: ", crime_data['New_Date'].max()) # Latest record of 2024
print("Min new_date value: ", crime_data['New_Date'].min()) # Earliest record of 2021

In [ ]:
crime_data = crime_data[crime_data['New_Date'].dt.year >= 2021]

In [ ]:
print("Min new_date value: ", crime_data['New_Date'].min()) # Earliest record of 2021
print(crime_data['New_Date'].dt.year.unique()) # Making sure that the range (2021-2024)

In [ ]:

crime_data.head()

## Columns that need to drop
X & Y Coordinate, Date, Block, IUCR, Description, Domestic, Beat, District, FBI code, Ward, Updated on, Latitude, Longitude 

In [ ]:
col = ['ID', 'New_Date', 'Primary Type', 'Location Description', 'Arrest', 'Community Area', 'Location']
crime_data = crime_data[col]
crime_data.info()

## Saving the Dataframe to a CSV file

In [ ]:
crime_data.to_csv('Crimes_2021_to_Present.csv')

In [ ]:
print("hello worlkd")